<a href="https://colab.research.google.com/github/yantvrs/Data_structure_2/blob/main/autocompleteWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import re
import time

class AVLNode:
    """
    A class representing a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        value: The value (word) stored in the node.
        left_child (AVLNode): The left child node.
        right_child (AVLNode): The right child node.
        height (int): The height of the subtree rooted at this node.
        imbalance (int): The imbalance factor of this node, calculated as
                         the difference between the heights of the left and
                         right subtrees.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value (str): The value (word) to be stored in this node.
        """
        self.value = value
        self.left_child = None
        self.right_child = None
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        self.height = 1 + max(left_height, right_height)
        self.imbalance = left_height - right_height


class AVLTree:
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.

    Attributes:
        root (AVLNode or None): The root node of the AVL tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the AVL Tree and rebalances the tree as needed.

        Args:
            value (str): The value (word) to be added to the tree.
        """
        self.root = self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the AVL tree.
        This method also ensures the tree remains balanced by updating node heights and performing rotations as needed.

        Args:
            current_node (AVLNode): The node to start the search for the insert position from.
            value (str): The value (word) to be added to the AVL tree.

        Returns:
            AVLNode: The node that either gets inserted or the node that was already present at that position.
        """

        if current_node is None:
            return AVLNode(value)

        if value < current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        current_node.calculate_height_and_imbalance()

        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """
        pivot = node.right_child
        node.right_child = pivot.left_child
        pivot.left_child = node

        node.calculate_height_and_imbalance()
        pivot.calculate_height_and_imbalance()

        return pivot

    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """
        pivot = node.left_child
        node.left_child = pivot.right_child
        pivot.right_child = node

        node.calculate_height_and_imbalance()
        pivot.calculate_height_and_imbalance()

        return pivot

    def _balance(self, node):
        """
        Balances the subtree rooted at the given node by performing rotations as needed.

        Args:
            node (AVLNode): The root node of the subtree that needs to be balanced.

        Returns:
            AVLNode: The new root node of the balanced subtree.

        Note:
            This method assumes that the height and imbalance factor of each node are up-to-date.
        """
        if node.imbalance == 2:
            pivot = node.left_child
            if pivot.imbalance == 1:
                return self._rotate_right(node)
            else:
                node.left_child = self._rotate_left(pivot)
                return self._rotate_right(node)
        else:
            pivot = node.right_child
            if pivot.imbalance == -1:
                return self._rotate_left(node)
            else:
                node.right_child = self._rotate_right(pivot)
                return self._rotate_left(node)

    def clean_text(self, text):
        """
        Removes punctuation and special characters from a text.

        Args:
            text (str): The text to be cleaned.

        Returns:
            str: The text without punctuation and special characters.
        """
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        return cleaned_text

    def insert_unique_words(self, text):
        """
        Inserts unique words from a text into the AVL tree.

        Args:
            text (str): The text containing words to be inserted.
        """
        text = text.lower()
        cleaned_text = self.clean_text(text)
        words = cleaned_text.split()
        unique_words = set(words)
        for word in unique_words:
            self.add(word)

    def find_words_with_prefix(self, prefix):
        """
        Finds words in the AVL tree that start with a given prefix.

        Args:
            prefix (str): The prefix to search for.

        Returns:
            list: A list of words that start with the given prefix.
        """
        prefix = self.clean_text(prefix).lower()
        completions = []
        self._find_words_with_prefix_recursive(self.root, prefix, completions)
        return completions

    def _find_words_with_prefix_recursive(self, node, prefix, completions):
        """
        Recursively finds words in the AVL tree that start with a given prefix.

        Args:
            node (AVLNode): The current node being checked.
            prefix (str): The prefix to search for.
            completions (list): The list to store found completions.
        """
        if node is None:
            return

        if node.value.lower().startswith(prefix):
            completions.append(node.value)

        if node.value.lower() >= prefix:
            self._find_words_with_prefix_recursive(node.left_child, prefix, completions)

        if node.value.lower() <= prefix:
            self._find_words_with_prefix_recursive(node.right_child, prefix, completions)

# Função para inserir palavras em uma lista
def insert_words_in_list(text):
    text = text.lower()
    cleaned_text = avl_tree.clean_text(text)
    words = cleaned_text.split()
    unique_words = set(words)
    return list(unique_words)

# Cria uma instância da árvore AVL
avl_tree = AVLTree()

# Corpus de palavras
words_corpus = """
A Revolução da Inteligência Artificial: Máquinas que Aprendem e Transformam o Mundo
A inteligência artificial (IA) é uma das inovações mais empolgantes e transformadoras do século XXI. Trata-se de um campo da ciência da computação que busca criar sistemas capazes de realizar tarefas que, até pouco tempo atrás, só podiam ser executadas por seres humanos, como raciocinar, tomar decisões, aprender e compreender linguagem natural. A IA representa a capacidade de as máquinas não apenas processarem dados, mas também de interpretá-los e tomar ações com base nessa interpretação.
Um dos principais catalisadores desse avanço tecnológico é o aprendizado de máquina (machine learning), um subcampo da IA que permite que os sistemas aprendam e melhorem a partir da experiência. Isso significa que, em vez de serem programadas para executar tarefas específicas, as máquinas podem aprender com dados e se adaptar a novas situações, tornando-se cada vez mais eficientes em suas operações.
A IA está presente em muitos aspectos da nossa vida cotidiana, mesmo que muitas vezes não a percebamos. Os mecanismos de recomendação em serviços de streaming, como Netflix e Spotify, são alimentados por algoritmos de IA que analisam nossas preferências e sugerem conteúdos relevantes. Os assistentes virtuais, como Siri, Google Assistant e Alexa, são exemplos de IA que podem responder a perguntas, realizar tarefas e até mesmo manter conversas naturais com os usuários.
Além do entretenimento e da assistência virtual, a IA tem uma ampla gama de aplicações em setores como medicina, finanças, transporte, manufatura e muito mais. Na medicina, por exemplo, a IA pode ser usada para diagnosticar doenças, analisar imagens médicas, prever surtos de epidemias e personalizar tratamentos com base nos dados do paciente. Nas finanças, os algoritmos de IA são usados para detecção de fraudes, previsões de mercado e gerenciamento de portfólio.
No entanto, a crescente influência da IA também levanta questões éticas e sociais importantes. Questões sobre privacidade de dados, viés algorítmico e o impacto da automação no emprego são apenas alguns dos desafios que a sociedade enfrenta à medida que a IA se torna cada vez mais onipresente. É essencial encontrar um equilíbrio entre a inovação tecnológica e a responsabilidade ética para garantir que a IA beneficie a humanidade como um todo.
À medida que a IA continua a evoluir, é emocionante pensar no que o futuro reserva. A IA promete revolucionar a maneira como vivemos, trabalhamos e interagimos com o mundo ao nosso redor. No entanto, é crucial que continuemos a estudar, debater e regulamentar o uso da IA para garantir que seus benefícios sejam maximizados e que os riscos sejam minimizados. Com a combinação certa de inovação e responsabilidade, a inteligência artificial tem o potencial de moldar um futuro mais inteligente e promissor para todos nós.
"""

# Remova pontuação e caracteres especiais.
words_corpus = words_corpus.lower()

# Remova pontuações e caracteres especiais do texto
cleaned_corpus = avl_tree.clean_text(words_corpus)

# Divida o texto limpo em palavras individuais
corpus_words = cleaned_corpus.split()

# Remover palavras duplicadas mantendo a ordem de ocorrência
corpus_words = list(dict.fromkeys(corpus_words))

# Inserir todas as palavras únicas do corpus na árvore AVL e medir o tempo de inserção
start_time = time.time()
avl_tree.insert_unique_words(words_corpus)
end_time = time.time()
print("Tempo de Inserção na Árvore AVL:", end_time - start_time, "segundos")

# Inserir todas as palavras únicas do corpus em uma lista e medir o tempo de inserção
start_time = time.time()
unique_words_list = insert_words_in_list(words_corpus)
end_time = time.time()
print("Tempo de Inserção na Lista:", end_time - start_time, "segundos")

# Comparar a altura da árvore AVL com o tamanho da lista
print("Altura da Árvore AVL:", avl_tree.root.height)
print("Tamanho da Lista:", len(unique_words_list))

# Encontrar palavras com um determinado prefixo na árvore AVL
prefix = "a"
start_time = time.time()
completions_avl = avl_tree.find_words_with_prefix(prefix)
end_time = time.time()
print(f"Tempo de Busca com Árvore AVL (Prefixo '{prefix}'): {end_time - start_time} segundos")
print(f"Palavras com o Prefixo '{prefix}':")
print(completions_avl)

# Encontrar palavras com um determinado prefixo na lista
prefix = "a"
start_time = time.time()
completions_list = [word for word in unique_words_list if word.startswith(prefix)]
end_time = time.time()
print(f"Tempo de Busca com Lista (Prefixo '{prefix}'): {end_time - start_time} segundos")
print(f"Palavras com o Prefixo '{prefix}':")
print(completions_list)

Tempo de Inserção na Árvore AVL: 0.0022170543670654297 segundos
Tempo de Inserção na Lista: 0.00025653839111328125 segundos
Altura da Árvore AVL: 10
Tamanho da Lista: 250
Tempo de Busca com Árvore AVL (Prefixo 'a'): 8.940696716308594e-05 segundos
Palavras com o Prefixo 'a':
['assistant', 'analisam', 'alguns', 'alexa', 'adaptar', 'a']
Tempo de Busca com Lista (Prefixo 'a'): 0.00012993812561035156 segundos
Palavras com o Prefixo 'a':
['aprendizado', 'atrs', 'analisam', 'assistentes', 'automao', 'adaptar', 'aplicaes', 'ampla', 'alm', 'alguns', 'avano', 'as', 'alimentados', 'assistant', 'at', 'aprender', 'aprendam', 'artificial', 'algortmico', 'alexa', 'assistncia', 'algoritmos', 'ao', 'aes', 'analisar', 'a', 'aprendem', 'aspectos', 'apenas']


A) Comparação de desempenho entre uma árvore AVL e uma estrutura de dados mais simples:

1. Tempo de Inserção:

A árvore AVL possui um tempo de inserção mais lento em comparação com uma lista ou uma árvore binária de busca simples. Isso ocorre porque a árvore AVL precisa realizar rotações e reequilíbrio após cada inserção para manter sua propriedade de balanceamento, o que adiciona alguma sobrecarga.
A lista tem o tempo de inserção mais rápido, pois não requer nenhum tipo de reequilíbrio ou reorganização.

2. Altura da Árvore:

A árvore AVL é projetada para manter uma altura balanceada, garantindo que a altura da árvore seja limitada pelo logaritmo do número de elementos na árvore. Portanto, a altura da árvore AVL é mais baixa e mais previsível em comparação com uma árvore binária de busca simples, que pode se tornar desequilibrada em casos de inserção desordenada.
A lista não é uma estrutura de árvore, portanto, não possui altura no sentido tradicional.

3. Tempo de Busca:

A árvore AVL tem um tempo de busca mais rápido do que uma lista não ordenada. Isso ocorre porque a árvore AVL mantém sua estrutura balanceada, garantindo um tempo de busca médio eficiente (O(log n)).
A árvore binária de busca não equilibrada pode ter um tempo de busca mais lento em comparação com a árvore AVL, especialmente em casos de inserção desordenada, onde pode se assemelhar a uma lista encadeada.
A lista não ordenada tem um tempo de busca O(n), onde n é o número de elementos na lista, o que pode ser significativamente mais lento do que a árvore AVL para grandes conjuntos de dados.

B) Impacto do tamanho do corpus no desempenho da árvore AVL:

1. Tempo de Inserção:

O tempo de inserção na árvore AVL aumentará à medida que o tamanho do corpus aumentar, devido à necessidade de reequilibrar a árvore após cada inserção. O aumento é aproximadamente O(log n), onde n é o número total de palavras únicas no corpus.

2. Altura da Árvore:

A altura da árvore AVL é controlada e deve permanecer próxima a O(log n), independentemente do tamanho do corpus. Portanto, o aumento no tamanho do corpus não deve afetar significativamente a altura da árvore AVL.

3. Tempo de Busca:

O tempo de busca na árvore AVL deve permanecer eficiente (O(log n)), mesmo com o aumento no tamanho do corpus, desde que a árvore esteja bem equilibrada.
No entanto, se o corpus contiver muitas palavras que compartilham um prefixo comum (por exemplo, um grande número de palavras que começam com "re" ou "un"), a árvore AVL pode ter um desempenho ligeiramente mais lento para buscar palavras com esses prefixos, devido à necessidade de percorrer subárvores maiores.

Em resumo, a árvore AVL é uma estrutura de dados eficiente para inserção e busca, mesmo com grandes corpos de texto, devido ao seu balanceamento automático. No entanto, o desempenho real dependerá da distribuição das palavras no corpus e da eficiência da implementação. Para conjuntos de dados extremamente grandes, outras estruturas, como árvores B ou hash tables, podem ser consideradas para otimização adicional.